In [2]:
import pandas as pd

In [3]:
train = pd.read_csv('./funda_train.csv')

In [5]:
train.head()

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000


In [6]:
temp = train[(train['transacted_date'] >= '2018-03-01') & (train['transacted_date'] <= '2018-05-31')]
temp_groupby = temp.groupby(['store_id'])['amount'].sum()

In [7]:
temp_groupby.head(5)

store_id
0    2.171000e+06
1    2.977143e+05
2    9.461429e+05
4    2.475643e+06
5    1.111857e+06
Name: amount, dtype: float64

In [117]:
a = [1, 2, 3, 4]

In [118]:
a[0]

1

In [119]:
s = pd.Series([1, 2, 3, 4], index=['A', 'B', 'C', 'D'])

In [120]:
s = {'A': 1, 'B':2, 'C':3, 'D':4}

In [121]:
s['A']

1

In [122]:
s['A']

1

In [123]:
type(temp_groupby.reset_index())

pandas.core.frame.DataFrame

In [124]:
baseline = temp_groupby.reset_index()

In [125]:
# list(train['store_id'].unique())

In [126]:
# list(baseline['store_id'].unique())

In [127]:
# train[train['store_id'] == 2119]

In [128]:
for i in [1, 2, 3]:
    print(i)

1
2
3


# 풀이1

In [129]:
set(train['store_id']) - set(baseline['store_id'])

{271, 279, 515, 588, 706, 795, 999, 1379, 1435, 2081, 2094, 2119}

# 풀이2

In [130]:
train[~train['store_id'].isin(list(baseline['store_id'].unique()))]['store_id'].unique()

array([ 271,  279,  515,  588,  706,  795,  999, 1379, 1435, 2081, 2094,
       2119], dtype=int64)

In [131]:
baseline_storeid = list(baseline['store_id'].unique())

In [132]:
#baseline_storeid

In [133]:
missing = train[~train['store_id'].isin(baseline_storeid)]

In [134]:
missing['store_id'].unique()

array([ 271,  279,  515,  588,  706,  795,  999, 1379, 1435, 2081, 2094,
       2119], dtype=int64)

# 풀이 3

In [135]:
train_id_list = list(train['store_id'].unique())
baseline_id_list = list(baseline['store_id'].unique())

In [136]:
missing = []
for i in train_id_list:
    if i not in baseline_id_list:
        missing.append(i)
print(missing)

[271, 279, 515, 588, 706, 795, 999, 1379, 1435, 2081, 2094, 2119]


# 새로운 아이디어

In [137]:
#1 (2019년 1월부터 2월까지 매출의 합 / 59(일)) * 92
#2 92일동안 월, 화, 수, 목, 금, 토, 일 갯수를 구해서 
#가지고 있는 데이터로 store_id 271번 월 50만원, 화 20만원, 수 60만원.... , 일 100만원
#92일 동안 월요일 13번, 화 12번, 수 13번... , 일 14번.
#50만원 * 13, 20만원 * 12 .... store_id 92일동안 = 

In [138]:
missing = set(train['store_id']) - set(baseline.index)
missing_df = train[train['store_id'].isin(list(missing))]

In [139]:
missing_predict = missing_df[(missing_df['transacted_date'] >= '2019-02-01') & (missing_df['transacted_date'] <= '2019-02-28')].groupby('store_id')['amount'].sum() * 92 / 28

In [140]:
#total_amout.reset_index()

In [141]:
train[train['store_id'].isin([0,1,2,4])]

,Unnamed: 0,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,2702,0,321,b,2018-01-03,11:39,0,NaN,기타 미용업,2000.000000
1,2703,0,1104,c,2018-01-03,12:03,0,NaN,기타 미용업,2000.000000
2,2704,0,1199,b,2018-01-03,12:39,0,NaN,기타 미용업,2000.000000
3,2705,0,927,g,2018-01-03,14:39,0,NaN,기타 미용업,7142.857143
4,2706,0,1200,c,2018-01-03,15:28,0,NaN,기타 미용업,1714.285714
5,2707,0,1201,a,2018-01-03,17:08,0,NaN,기타 미용업,1857.142857
6,2708,0,1202,a,2018-01-03,19:39,0,NaN,기타 미용업,20000.000000
7,2709,0,992,c,2018-01-04,15:09,0,NaN,기타 미용업,6428.571429
8,2710,0,1169,h,2018-01-04,17:02,0,NaN,기타 미용업,1857.142857
9,2711,0,1203,a,2018-01-05,14:04,3,NaN,기타 미용업,11428.571429


In [142]:
#내가 했던 코딩_이건...데이터 연결이 아닌 리스트 연결이기 때문에 부적합.

In [143]:
#store_id = list(set(train['store_id']))

In [144]:
#for j in (0,len(missing)):
#    if j not in store_id:




#        total_amount.append(j)
#print(total_amout)

In [145]:
pd.concat([missing_predict, baseline]).to_csv('./sub20190814.csv')

In [146]:
#pd.concat([exceptions_submission, submission]).to_csv('./submission.csv')